In [437]:
import torchvision
from PIL import Image

In [438]:
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import numpy as np
import torch

In [439]:
from torchvision.datasets import MNIST

In [440]:
try:
    import accimage
except ImportError:
    accimage = None
    

def _is_pil_image(img):
    if accimage is not None:
        return isinstance(img, (Image.Image, accimage.Image))
    else:
        return isinstance(img, Image.Image)
    
    
def _rgb2hsv(testing):
    max_vals, _ = torch.max(testing, dim=0)
    min_vals, _ = torch.min(testing, dim=0)
    v = max_vals
    s = (max_vals - min_vals) / max_vals
    df = max_vals - min_vals
    r, g, b = testing[0], testing[1], testing[2]
    
    
    h = max_vals != min_vals
    hr =  (max_vals == r) * ((g - b) / df) 
    hg =  (max_vals == g) * ((b - r) / df + 2.0)
    hb =  (max_vals == b) * ((r - g) / df + 4.0)    
    h = h * (hr + hg + hb)
    h = (h / 6.0) % 1.0
    h[h != h] = 0
    return torch.stack((h, s, v))
 
def _hsv2rgb(testing):
    h, s, v = testing[0], testing[1], testing[2]
    i = (h * 6.0).type(torch.IntTensor)
    f = (h * 6.0) - i
    p = v * (1.0 - s)
    q = v * (1.0 - f * s)
    t = v * (1.0 - (1 - f) * s)
    i = i % 6
    r = (i == 0) * v + (i == 1) * q +  (i == 2) * p + (i == 3) * p + (i == 4) * t + (i == 5) * v 
    g = (i == 0) * t + (i == 1) * v +  (i == 2) * v + (i == 3) * q + (i == 4) * p + (i == 5) * p
    b = (i == 0) * p + (i == 1) * p +  (i == 2) * t + (i == 3) * v + (i == 4) * v + (i == 5) * q
    return torch.stack((r, g, b))
    
def adjust_hue2(img, hue_factor):
    """Adjust hue of an image.

    The image hue is adjusted by converting the image to HSV and
    cyclically shifting the intensities in the hue channel (H).
    The image is then converted back to original image mode.

    `hue_factor` is the amount of shift in H channel and must be in the
    interval `[-0.5, 0.5]`.

    See `Hue`_ for more details.

    .. _Hue: https://en.wikipedia.org/wiki/Hue

    Args:
        img (PIL Image or torch.Tensor): Image to be adjusted.
                                         Input can be PIL or torch.Tensor.
        hue_factor (float):  How much to shift the hue channel. Should be in
            [-0.5, 0.5]. 0.5 and -0.5 give complete reversal of hue channel in
            HSV space in positive and negative direction respectively.
            0 means no shift. Therefore, both -0.5 and 0.5 will give an image
            with complementary colors while 0 gives the original image.

    Returns:
        PIL Image or torch.Tensor: Hue adjusted image.
        If input is PIL Image, return PIL Image
        If input is torch.Tensor, return torch.Tensor
    """
    if not(-0.5 <= hue_factor <= 0.5):
        raise ValueError('hue_factor is not in [-0.5, 0.5].'.format(hue_factor))
    
    if not _is_pil_image(img) and type(img) is not torch.Tensor:
        raise TypeError('img should be PIL Image or torch.Tensor. Got {}'.format(type(img)))
        
    if _is_pil_image(img):
        print("pil working")

        input_mode = img.mode
        if input_mode in {'L', '1', 'I', 'F'}:
            return img

        h, s, v = img.convert('HSV').split()
        tmp_h = transform(h)
        np_h = np.array(h, dtype=np.uint8)
        # uint8 addition take cares of rotation across boundaries
        with np.errstate(over='ignore'):
            np_h += np.uint8(hue_factor * 255)
        h = Image.fromarray(np_h, 'L')
        tmp = transform(h)
        img = Image.merge('HSV', (h, s, v)).convert(input_mode)
        
        return img
    else:
        assert type(img) is torch.Tensor
        assert len(img.shape) == 3 # input img must be 3D torch.Tensor
        assert img.shape[0] == 3 # input img must have 3 channels 
        # the default ToTensor in torchvision scale the RGB from [0,255] to [0, 1]
        img = _rgb2hsv(img)
        h, s, v = img[0], img[1], img[2]
        new_h = h * 255
        new_h = (new_h).type(torch.IntTensor)
        new_h += int(hue_factor * 255)

        new_h = new_h.type(torch.FloatTensor)
        new_h = new_h / 255.0
        new_img = _hsv2rgb(torch.stack((new_h, s, v)))
        return new_img

In [441]:
# open a new image
img_cat = Image.open("/Users/xni/Documents/pytorch_task/cat.jpg")
# img_cat.show()

In [442]:
# use the old adjust_hue 
# when input is PIL image, we use the old adjust_hue
img_new = adjust_hue2(img_cat, 0.5)


pil working


In [443]:
# show the image adjusted using old method
# img_new.show()

In [444]:
# transform the image into torch.Tensor
transform = transforms.ToTensor()
img_cat_tensor = transform(img_cat)
img_cat_tensor.shape

torch.Size([3, 559, 838])

In [445]:
# use the new adjust_hue
# when input is torch.Tensor, we use the new adjust_hue
new_cat = adjust_hue2(img_cat_tensor, 0.5)

In [446]:
# transform the image back into PIL image
transform2 = transforms.ToPILImage()
img_cal_pil = transform2(new_cat)

In [447]:
# show the image adjusted using new method

# img_cal_pil.show()

In [448]:
import unittest
import torchvision.transforms.functional as F

In [449]:
class Tester(unittest.TestCase):
    def test_adjust_hue(self):
        x_shape = [2, 2, 3]
        x_data = [0, 5, 13, 54, 135, 226, 37, 8, 234, 90, 255, 1]
        x_np = np.array(x_data, dtype=np.uint8).reshape(x_shape)
        x_pil = Image.fromarray(x_np, mode='RGB')
        
        transform = transforms.ToTensor()
        x_tensor = transform(x_pil)
        with self.assertRaises(ValueError):
            F.adjust_hue(x_pil, -0.7)
            F.adjust_hue(x_pil, 1)
            
            adjust_hue2(x_tensor, -0.7)
            adjust_hue2(x_tensor, 1)

        # test 0: almost same as x_data but not exact.
        # probably because hsv <-> rgb floating point ops
        
        # original test 
        y_pil = F.adjust_hue(x_pil, 0)
        y_np = np.array(y_pil)

        # test for torch.Tensor
        y_tensor = adjust_hue2(x_tensor, 0)
        y_tensor_after = torch.round(y_tensor * 255).type(torch.IntTensor).permute(1, 2, 0)
        y_tensor_np = y_tensor_after.numpy()

        y_ans = [0, 5, 13, 54, 139, 226, 35, 8, 234, 91, 255, 1]
        y_ans = np.array(y_ans, dtype=np.uint8).reshape(x_shape)
        
        assert np.allclose(y_np, y_ans)
        assert np.allclose(y_tensor_np, y_ans)


        
        # test 1
        # original test 
        y_pil = adjust_hue2(x_pil, 0.25)
        y_np = np.array(y_pil)
        
        # test for torch.Tensor
        y_tensor = adjust_hue2(x_tensor, 0.25)
        y_tensor_after = torch.round(y_tensor * 255).type(torch.IntTensor).permute(1, 2, 0)
        y_tensor_np = y_tensor_after.numpy()
        print("=====y_np:", y_np.shape, y_np)
        print("=====y_tensor_np:",y_tensor_np)


        
        y_ans = [13, 0, 12, 224, 54, 226, 234, 8, 99, 1, 222, 255]
        y_ans = np.array(y_ans, dtype=np.uint8).reshape(x_shape)
        assert np.allclose(y_np, y_ans)
        assert np.allclose(y_tensor_np, y_ans)




        # test 2
        # original test 
        y_pil = F.adjust_hue(x_pil, -0.25)
        y_np = np.array(y_pil)
        
        # test for torch.Tensor
        y_tensor = adjust_hue2(x_tensor, -0.25)
        y_tensor_after = torch.round(y_tensor * 255).type(torch.IntTensor).permute(1, 2, 0)
        y_tensor_np = y_tensor_after.numpy()

        y_ans = [0, 13, 2, 54, 226, 58, 8, 234, 152, 255, 43, 1]
        y_ans = np.array(y_ans, dtype=np.uint8).reshape(x_shape)
        assert np.allclose(y_np, y_ans)
        assert np.allclose(y_tensor_np, y_ans)



In [450]:
t = Tester()

In [451]:
t.test_adjust_hue()

pil working
=====y_np: (2, 2, 3) [[[ 13   0  12]
  [224  54 226]]

 [[234   8  99]
  [  1 222 255]]]
=====y_tensor_np: [[[ 13   0  12]
  [224  54 226]]

 [[234   8  98]
  [  1 222 255]]]


AssertionError: 

# Testing the rgb2hsv and hsv2rgb

In [452]:
# prepare the testing data
x_shape = [2, 2, 3]
x_data = [0, 5, 13, 54, 135, 226, 37, 8, 234, 90, 255, 1]
x_np = np.array(x_data, dtype=np.uint8).reshape(x_shape)
x_pil = Image.fromarray(x_np, mode='RGB')
        
transform = transforms.ToTensor()
x_tensor = transform(x_pil)

In [466]:
# PIL: convert from RGB to HSV
h, s, v = x_pil.convert('HSV').split()
print(transform(h))
print(transform(s))
print(transform(v))

tensor([[[0.6000, 0.5843],
         [0.6863, 0.2745]]])
tensor([[[1.0000, 0.7608],
         [0.9647, 0.9961]]])
tensor([[[0.0510, 0.8863],
         [0.9176, 1.0000]]])


In [463]:
# PIL: convert from RGB to HSV

new_img = Image.merge('HSV', (h, s, v)).convert("RGB")
new_img_tensor = transform(new_img)
new_img_tensor

tensor([[[0.0000, 0.2118],
         [0.1373, 0.3569]],

        [[0.0196, 0.5451],
         [0.0314, 1.0000]],

        [[0.0510, 0.8863],
         [0.9176, 0.0039]]])

In [454]:
# Our: convert from RGB to HSV
_hsv2rgb(torch.stack((transform(h), transform(s), transform(v))))

tensor([[[[0.0000, 0.2120],
          [0.1365, 0.3555]]],


        [[[0.0204, 0.5452],
          [0.0324, 1.0000]]],


        [[[0.0510, 0.8863],
          [0.9176, 0.0039]]]])

In [455]:
# PIL: convert from HSV to RGB
h, s, v = x_pil.convert('HSV').split()
transform(h), transform(s), transform(v)

(tensor([[[0.6000, 0.5843],
          [0.6863, 0.2745]]]), tensor([[[1.0000, 0.7608],
          [0.9647, 0.9961]]]), tensor([[[0.0510, 0.8863],
          [0.9176, 1.0000]]]))

In [456]:
# Our: convert from HSV to RGB
_rgb2hsv(x_tensor)

tensor([[[0.6026, 0.5882],
         [0.6881, 0.2749]],

        [[1.0000, 0.7611],
         [0.9658, 0.9961]],

        [[0.0510, 0.8863],
         [0.9176, 1.0000]]])

# Another method to convert RGB and HSV

In [457]:
import colorsys
print(colorsys.rgb_to_hsv(0.2157, 0.2118, 0.2667))
_rgb2hsv(torch.stack((torch.Tensor([0.2157]).view(1, 1, 1), torch.Tensor([0.2118]).view(1, 1, 1), torch.Tensor([0.2667]).view(1, 1, 1))))

(0.6785063752276868, 0.20584926884139484, 0.2667)


tensor([[[[0.6785]]],


        [[[0.2058]]],


        [[[0.2667]]]])

In [458]:
print(colorsys.hsv_to_rgb(0.6785063752276868, 0.20584926884139484, 0.2667))
_hsv2rgb(torch.stack((torch.Tensor([0.6785063752276868]).view(1, 1, 1), torch.Tensor([0.20584926884139484]).view(1, 1, 1), torch.Tensor([0.2667]).view(1, 1, 1))))

(0.2157, 0.2118, 0.2667)


tensor([[[[0.2157]]],


        [[[0.2118]]],


        [[[0.2667]]]])

In [467]:
print(colorsys.hsv_to_rgb(0.6000, 1.0000, 0.0510))
_hsv2rgb(torch.stack((torch.Tensor([0.6000]).view(1, 1, 1), torch.Tensor([1.0000]).view(1, 1, 1), torch.Tensor([0.0510]).view(1, 1, 1))))

(0.0, 0.020400000000000015, 0.051)


tensor([[[[0.0000]]],


        [[[0.0204]]],


        [[[0.0510]]]])